In [1]:
pip install pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\keert\anaconda3\python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd

# Load your dataset into a Pandas DataFrame
df = pd.read_csv(r'C:\Users\keert\OneDrive\Desktop\Shopping_Deals_Recommeneder\dataset\user_item_ratings.csv')

# Check the first few rows to understand the data structure
print(df.head())


   user_id     item_id  rating
0      101  Item_101_1     4.6
1      101  Item_101_2     4.6
2      101  Item_101_3     4.6
3      101  Item_101_4     4.6
4      101  Item_101_5     4.6


In [6]:
# Check for missing values in the dataset
print(df.isnull().sum())  # This shows the count of missing values in each column


user_id    0
item_id    0
rating     0
dtype: int64


In [7]:
# Check the data types of each column
print(df.dtypes)


user_id      int64
item_id     object
rating     float64
dtype: object


In [8]:
# Convert 'user_id' and 'item_id' to string (if they aren't already)
df['user_id'] = df['user_id'].astype(str)
df['item_id'] = df['item_id'].astype(str)

# Convert 'rating' to float (if it isn't already)
df['rating'] = df['rating'].astype(float)

# Check the data types again to ensure they've been updated
print(df.dtypes)


user_id     object
item_id     object
rating     float64
dtype: object


In [9]:
# Create a user-item matrix (pivot table)
user_item_matrix = df.pivot(index='user_id', columns='item_id', values='rating')

# Display the user-item matrix
print(user_item_matrix.head())


item_id  Item_101_1  Item_101_10  Item_101_11  Item_101_12  Item_101_13  \
user_id                                                                   
101             4.6          4.6          4.6          4.6          4.6   
102             NaN          NaN          NaN          NaN          NaN   
103             NaN          NaN          NaN          NaN          NaN   
104             NaN          NaN          NaN          NaN          NaN   
105             NaN          NaN          NaN          NaN          NaN   

item_id  Item_101_14  Item_101_2  Item_101_3  Item_101_4  Item_101_5  ...  \
user_id                                                               ...   
101              4.6         4.6         4.6         4.6         4.6  ...   
102              NaN         NaN         NaN         NaN         NaN  ...   
103              NaN         NaN         NaN         NaN         NaN  ...   
104              NaN         NaN         NaN         NaN         NaN  ...   
105         

In [10]:
user_item_matrix = user_item_matrix.fillna(user_item_matrix.mean())


In [11]:
global_mean = user_item_matrix.stack().mean()
user_item_matrix = user_item_matrix.fillna(global_mean)


In [12]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Prepare data for Surprise
reader = Reader(rating_scale=(1, 5))  # Assuming ratings are between 1 and 5
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)

# Split the data into train and test sets (80% training, 20% testing)
trainset, testset = train_test_split(data, test_size=0.2)

# Initialize the SVD model
svd = SVD()

# Train the model on the training set
svd.fit(trainset)

# Evaluate the model using RMSE (Root Mean Squared Error)
predictions = svd.test(testset)
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")


RMSE: 0.2794
RMSE: 0.2794477251423484


In [13]:
import pandas as pd

# Load the ratings data
ratings_data_path = "user_item_ratings.csv"  # Replace with your actual path
ratings_df = pd.read_csv(ratings_data_path)

# Check if the user_id exists in the dataset
user_id_to_check = 107  # Replace with the user_id you want to check

# Check if the user_id exists in the 'user_id' column
user_exists = ratings_df['user_id'].isin([user_id_to_check]).any()

if user_exists:
    print(f"User ID {user_id_to_check} exists in the dataset.")
else:
    print(f"User ID {user_id_to_check} does NOT exist in the dataset.")


User ID 107 exists in the dataset.


In [14]:
# Check if the user exists in the model's internal mapping
if str(user_id_to_check) in recommender.trainset._raw2inner_id_users:
    print(f"User ID {user_id_to_check} exists in the model's internal mapping.")
else:
    print(f"User ID {user_id_to_check} does NOT exist in the model's internal mapping.")


NameError: name 'recommender' is not defined

In [15]:
user_ratings = ratings_df[ratings_df['user_id'] == user_id_to_check]
print(user_ratings)


    user_id      item_id  rating
74      107   Item_107_1     4.5
75      107   Item_107_2     4.5
76      107   Item_107_3     4.5
77      107   Item_107_4     4.5
78      107   Item_107_5     4.5
79      107   Item_107_6     4.5
80      107   Item_107_7     4.5
81      107   Item_107_8     4.5
82      107   Item_107_9     4.5
83      107  Item_107_10     4.5
84      107  Item_107_11     4.5
85      107  Item_107_12     4.5
86      107  Item_107_13     4.5
87      107  Item_107_14     4.5
88      107  Item_107_15     4.5


In [16]:
import joblib

# Load the trained collaborative filtering model
model = joblib.load('collaborative_model.pkl')

# Access the trainset from the model
trainset = model.trainset

# Check if User ID 107 exists in the raw-to-internal user ID mapping
user_id = 107

# Check if the user_id exists in the model's internal training data
if str(user_id) in trainset._raw2inner_id_users:
    print(f"User ID {user_id} exists in the model's training data!")
else:
    print(f"User ID {user_id} does NOT exist in the model's training data.")


User ID 107 does NOT exist in the model's training data.


In [17]:
import joblib

# Load the trained collaborative filtering model
model = joblib.load('collaborative_model.pkl')

# Access the trainset from the model
trainset = model.trainset

# Get all the raw user IDs that were present in the training data
raw_user_ids = list(trainset._raw2inner_id_users.keys())

# Print the list of raw user IDs
print("User IDs in the model's training data:")
print(raw_user_ids)


User IDs in the model's training data:
[101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 

In [18]:
user_id = 107
if str(user_id) in raw_user_ids:
    print(f"User ID {user_id} exists in the model's training data.")
else:
    print(f"User ID {user_id} does NOT exist in the model's training data.")


User ID 107 does NOT exist in the model's training data.


In [19]:
from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy

# Load the complete dataset
data = Dataset.load_from_df(ratings_df[['user_id', 'item_id', 'rating']], Reader(rating_scale=(1, 5)))

# Train-test split (optional, for validation)
trainset = data.build_full_trainset()

# Initialize and train the model
model = SVD()
model.fit(trainset)

# Save the model
joblib.dump(model, 'collaborative_model_updated.pkl')


['collaborative_model_updated.pkl']

In [20]:
if str(user_id) in raw_user_ids:
    internal_user_id = trainset._raw2inner_id_users[str(user_id)]
    recommendations = model.get_recommendations(internal_user_id)
else:
    # Fallback: Recommend top-rated items
    recommendations = get_top_rated_items(ratings_df)  # Implement this function based on ratings_df


NameError: name 'get_top_rated_items' is not defined